In [ ]:
from IPython.core.display import HTML
with open('../style.css', 'r') as file:
    css = file.read()
HTML(css)

# Saving the Infidels

In this notebook we want so solve a famous search problem, which is usually known as the
[missionaries and cannibals problem](https://en.wikipedia.org/wiki/Missionaries_and_cannibals_problem):
Three missinaries and three infidels have to cross a river in order to get to a church where the infidels can be baptized.  In order to cross the river, they have to take a small boat that can take at most two passengers.  If at any moments at any shore there are more infidels than missionaries, then the missionaries have a problem, since the infidels have a diet that includes human flesh.

We will encode this problem as a *constraint satisfaction problem*.  In order to do so, we assume that the
problem can be solved with $n\in\mathbb{N}$ crossing of the river.  We use the following variables:
* $\texttt{M}i$ for $i\in\{0,\cdots,n\}$ is the number of missionaries on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{C}i$ for $i\in\{0,\cdots,n\}$ is the number of infidels on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{B}i$ for $i\in\{0,\cdots,n\}$ is the number of boats on the western shore after the 
  $i^{\textrm{th}}$ crossing.

In [ ]:
import z3

## Auxiliary Functions

The function `start` takes three `Z3` variables as input:
* `M` is the number of missionaries on the western shore,
* `C` is the number of infidels on the western shore,
* `B` is the number of boats on the western shore.

It returns a formual that specifies that everybody is on the western shore.

In [ ]:
def start(M, C, B):
    return z3.And(M == 3, C == 3, B == 1)

The function `goal` takes three `Z3` variables as input:
* `M` is the number of missionaries on the western shore,
* `C` is the number of infidels on the western shore,
* `B` is the number of boats on the western shore.

It returns a formual that specifies that everybody is on the eastern shore.

In [ ]:
def goal(M, C, B):
    return z3.And(M == 0, C == 0, B == 0)

The function `invariant` takes three `Z3` as input:
* `M` is the number of missionaries on the western shore,
* `C` is the number of infidels on the western shore,
* `B` is the number of boats on the western shore.

It returns a formula that is `True` if there is no problem on either shore of the river.  There is no problem if any of the following conditions is true:
* There are no missionaries on the western side of the shore, i.e. 
  $\texttt{M} = 0$.  
  Then all missionaries are on the eastern side of the shore.
* All missionaries are on the western side of the shore, i.e. $\texttt{M} = 3$.
  Then there are no missionaries on the eastern side of the shore.
* The number of missionaries on the western side is the same as the number of 
  infidels on that side, i.e. $\texttt{M} = \texttt{C}$.  Then the numbers of 
  missionaries and infidels have to match on the eastern shore as well.

In [ ]:
def invariant(M, C, B):
    return z3.Or(M == 0, M == 3, M == C)

The function `transition` takes 6 arguments:
* `M𝛼` is the number of missionaries on the eastern shore before the crossing.
* `C𝛼` is the number of infidels on the eastern shore before the crossing.
* `B𝛼` is the number of boats on the eastern shore before the crossing. 
* `M𝛽` is the number of missionaries on the eastern shore after the crossing.
* `C𝛽` is the number of infidels on the eastern shore after the crossing.
* `B𝛽` is the number of infidels on the eastern shore after the crossing.

The function returns a set of formulas that is `True` if the missionaries starting on one shore arrive at the opposite  shore after the $i^{\textrm{th}}$ crossing.  Note that if $i$ is odd, then during the $i^{\textrm{th}}$ crossing the boat travels from the western shore to the eastern shore.  If $i$ is even, the boat travels from the eastern shore to the western shore.

In [ ]:
def transition(M𝛼, C𝛼, B𝛼, M𝛽, C𝛽, B𝛽):
    Formulas  = { B𝛽 == 1 - B𝛼 }
    Formulas |= { z3.Implies(B𝛼 == 1, 
                             z3.And(1 <= M𝛼 - M𝛽 + C𝛼 - C𝛽, 
                                    2 >= M𝛼 - M𝛽 + C𝛼 - C𝛽,
                                    M𝛽 <= M𝛼, 
                                    C𝛽 <= C𝛼)
                            ),
                  z3.Implies(B𝛼 == 0, 
                             z3.And(1 <= M𝛽 - M𝛼 + C𝛽 - C𝛼,
                                    2 >= M𝛽 - M𝛼 + C𝛽 - C𝛼,
                                    M𝛽 >= M𝛼, 
                                    C𝛽 >= C𝛼)
                            )
                }
    return z3.And(*Formulas)

The function `missionaries_CSP` creates a CSP that tries to solve the problem with `n` crossings.

In [ ]:
def missionaries_CSP(n):
    S = z3.Solver()
    Ms = [z3.Int(f'M{i}') for i in range(n+1)]
    Is = [z3.Int(f'C{i}') for i in range(n+1)]
    Bs = [z3.Int(f'B{i}') for i in range(n+1)]
    Constraints  = { start(Ms[0], Is[0], Bs[0]) }  # start state
    Constraints |= { goal( Ms[n], Is[n], Bs[n]) }  # goal state
    for i in range(n):
        Constraints.add(invariant(Ms[i], Is[i], Bs[i]))
        Constraints.add(transition(Ms[i  ], Is[i  ], Bs[i  ], 
                                   Ms[i+1], Is[i+1], Bs[i+1]))
        Constraints.add(0 <= Ms[i])
        Constraints.add(0 <= Is[i])
        Constraints.add(0 <= Bs[i])
        Constraints.add(Ms[i] <= 3)
        Constraints.add(Is[i] <= 3)
        Constraints.add(Bs[i] <= 1)
    S.add(Constraints)
    result = str(S.check())
    if result == 'sat':
        Model = S.model()
        Solution = (   { f'M{i}': Model[Ms[i]] for i in range(n+1) }
                     | { f'C{i}': Model[Is[i]] for i in range(n+1) }
                     | { f'B{i}': Model[Bs[i]] for i in range(n+1) }
                   )
        return { key: Solution[key].as_long() for key in Solution }            
    else:
        return None

The function `find_solution` computes a solution to the problem of saving the infidels.

In [ ]:
def find_solution():
    n = 1
    while True:
        print(n)
        Solution = missionaries_CSP(n)
        if Solution != None:
            return n, Solution
        n += 2

On my desktop computer (2017 iMac with 3.4 GHz Quad-Core Intel i5) it takes about 2 seconds to solve the problem. 

In [ ]:
%%time
n, Solution = find_solution()
n, Solution

In [ ]:
def show_solution(Solution, n):
    for i in range(n+1):
        M = Solution[f'M{i}']
        C = Solution[f'C{i}']
        B = Solution[f'B{i}']
        print('😇' * M + '🥷' * C + ' ' * 28 + '😇' * (3 - M) + '🥷' * (3 - C))
        if B == 1:
            MB = Solution[f'M{i}'] - Solution[f'M{i+1}']
            CB = Solution[f'C{i}'] - Solution[f'C{i+1}']
            print(' ' * 12 + '>>> ' + '😇'*MB + '🥷'*CB + ' >>>')
        elif i + 1 < n:
            MB = Solution[f'M{i+1}'] - Solution[f'M{i}']
            CB = Solution[f'C{i+1}'] - Solution[f'C{i}']
            print(' ' * 12 + '<<< ' + '😇'*MB + '🥷'*CB + ' <<<')

In [ ]:
show_solution(Solution, n)